# 池化层
回忆⼀下，在[“⼆维卷积层”](5.1cov-layer.ipynb) ⼀节⾥介绍的图像物体边缘检测应⽤中，我们构造卷积核从而精确
地找到了像素变化的位置。设任意⼆维数组X的i⾏j列的元素为X[i, j]。如果我们构造的卷积
核输出Y[i, j]=1，那么说明输⼊中X[i, j]和X[i, j+1]数值不⼀样。这可能意味着物体边
缘通过这两个元素之间。但实际图像⾥，我们感兴趣的物体不会总出现在固定位置：即使我们连
续拍摄同⼀个物体也极有可能出现像素位置上的偏移。这会导致同⼀个边缘对应的输出可能出现
在卷积输出Y中的不同位置，进而对后⾯的模式识别造成不便。

在本节中我们介绍池化（pooling）层，它的提出是为了缓解卷积层对位置的过度敏感性。
## ⼆维最⼤池化层和平均池化层
同卷积层⼀样，池化层每次对输⼊数据的⼀个固定形状窗口（⼜称池化窗口）中的元素计算输出。
不同于卷积层⾥计算输⼊和核的互相关性，池化层直接计算池化窗口内元素的最⼤值或者平均
值。该运算也分别叫做最⼤池化或平均池化。在⼆维最⼤池化中，池化窗口从输⼊数组的最左上
⽅开始，按从左往右、从上往下的顺序，依次在输⼊数组上滑动。当池化窗口滑动到某⼀位置时，
窗口中的输⼊⼦数组的最⼤值即输出数组中相应位置的元素。

![池化窗口形状为$2\\times 2$的最大池化](../img/pooling.svg)

图5.6展⽰了池化窗口形状为2 × 2的最⼤池化，阴影部分为第⼀个输出元素及其计算所使⽤的输
⼊元素。输出数组的⾼和宽分别为2，其中的4个元素由取最⼤值运算max得出：
⼆维平均池化的⼯作原理与⼆维最⼤池化类似，但将最⼤运算符替换成平均运算符。池化窗口形
状为p × q的池化层称为p × q池化层，其中的池化运算叫作p × q池化。

让我们再次回到本节开始提到的物体边缘检测的例⼦。现在我们将卷积层的输出作为2 × 2最⼤
池化的输⼊。设该卷积层输⼊是X、池化层输出为Y。⽆论是X[i, j]和X[i, j+1]值不同，还
是X[i, j+1]和X[i, j+2]不同，池化层输出均有Y[i, j]=1。也就是说，使⽤2 × 2最⼤池化
层时，只要卷积层识别的模式在⾼和宽上移动不超过⼀个元素，我们依然可以将它检测出来。

下⾯把池化层的前向计算实现在pool2d函数⾥。它跟“⼆维卷积层” ⼀节⾥corr2d函数⾮常类
似，唯⼀的区别在计算输出Y上。

In [2]:
from mxnet import nd
from mxnet.gluon import nn
def pool2d(X, pool_size, mode='max'):
    p_h, p_w = pool_size
    Y = nd.zeros((X.shape[0] - p_h + 1, X.shape[1] - p_w + 1))
    for i in range(Y.shape[0]):
        for j in range(Y.shape[1]):
            if mode == 'max':
                Y[i, j] = X[i: i + p_h, j: j + p_w].max()
            elif mode == 'avg':
                Y[i, j] = X[i: i + p_h, j: j + p_w].mean()
    return Y

我们可以构造图5.6中的输⼊数组X来验证⼆维最⼤池化层的输出。

In [4]:
X = nd.array([[0,1,2],[3,4,5],[6,7,8]])
pool2d(X,(2,2))


[[4. 5.]
 [7. 8.]]
<NDArray 2x2 @cpu(0)>

同时我们实验⼀下平均池化层。

In [5]:
pool2d(X, (2,2),'avg')


[[2. 3.]
 [5. 6.]]
<NDArray 2x2 @cpu(0)>

## 填充和步幅
同卷积层⼀样，池化层也可以在输⼊的⾼和宽两侧的填充并调整窗口的移动步幅来改变输出形
状。池化层填充和步幅与卷积层填充和步幅的⼯作机制⼀样。我们将通过nn模块⾥的⼆维最⼤池
化层MaxPool2D来演⽰池化层填充和步幅的⼯作机制。我们先构造⼀个形状为(1, 1, 4, 4)的输⼊
数据，前两个维度分别是批量和通道。

In [12]:
X = nd.arange(16).reshape(1,1,4,4)
X


[[[[ 0.  1.  2.  3.]
   [ 4.  5.  6.  7.]
   [ 8.  9. 10. 11.]
   [12. 13. 14. 15.]]]]
<NDArray 1x1x4x4 @cpu(0)>

默认情况下， MaxPool2D实例⾥步幅和池化窗口形状相同。下⾯使⽤形状为(3, 3)的池化窗口，默
认获得形状为(3, 3)的步幅

In [7]:
pool2d = nn.MaxPool2D(3)
pool2d(X)


[[[[10.]]]]
<NDArray 1x1x1x1 @cpu(0)>

我们可以⼿动指定步幅和填充。

In [8]:
pool2d = nn.MaxPool2D(3,2,1)
pool2d(X)


[[[[ 5.  7.]
   [13. 15.]]]]
<NDArray 1x1x2x2 @cpu(0)>

当然，我们也可以指定⾮正⽅形的池化窗口，并分别指定⾼和宽上的填充和步幅。

In [9]:
pool2d = nn.MaxPool2D((2, 3), padding=(1, 2), strides=(2, 3))
pool2d(X)


[[[[ 0.  3.]
   [ 8. 11.]
   [12. 15.]]]]
<NDArray 1x1x3x2 @cpu(0)>

## 多通道
在处理多通道输⼊数据时，池化层对每个输⼊通道分别池化，而不是像卷积层那样将各通道的输
⼊按通道相加。这意味着池化层的输出通道数与输⼊通道数相等。下⾯将数组X和X+1在通道维
上连结来构造通道数为2的输⼊。

In [13]:
X = nd.concat(X,X+1,dim = 1)
X


[[[[ 0.  1.  2.  3.]
   [ 4.  5.  6.  7.]
   [ 8.  9. 10. 11.]
   [12. 13. 14. 15.]]

  [[ 1.  2.  3.  4.]
   [ 5.  6.  7.  8.]
   [ 9. 10. 11. 12.]
   [13. 14. 15. 16.]]]]
<NDArray 1x2x4x4 @cpu(0)>

池化后，我们发现输出通道数仍然是2。

In [15]:
pool2d = nn.MaxPool2D(3, padding=1, strides=2)
pool2d(X)


[[[[ 5.  7.]
   [13. 15.]]

  [[ 6.  8.]
   [14. 16.]]]]
<NDArray 1x2x2x2 @cpu(0)>

## 小结
- 最⼤池化和平均池化分别取池化窗口中输⼊元素的最⼤值和平均值作为输出。
- 池化层的⼀个主要作⽤是缓解卷积层对位置的过度敏感性。
- 可以指定池化层的填充和步幅。
- 池化层的输出通道数跟输⼊通道数相同。